In [1]:
# !pip install tensorflow_hub
# !pip install tfswin

In [2]:
from keras.preprocessing.image import ImageDataGenerator

from keras.applications import Xception

from keras.models import Model

from keras.layers import Dense, Flatten , Input , Lambda

from keras.optimizers import Adam

from keras.callbacks import ReduceLROnPlateau

train_dir = '/common/scratch/projectgrps/CS424/assets/20230328/test_1/train_'

datagen = ImageDataGenerator(
    
    rescale=1./255,
    
    validation_split=0.20,
    
    rotation_range=20,
    
    width_shift_range=0.2,
    
    height_shift_range=0.2,
    
    shear_range=0.2,
    
    zoom_range=0.2,
    
    horizontal_flip=True,
    
    fill_mode='nearest'
)

batch_size = 32 

num_classes = 59

train_generator = datagen.flow_from_directory(
    
    train_dir,
    
    target_size=(224, 224),
    
    batch_size=batch_size,
    
    class_mode='categorical',
    
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    
    train_dir,
    
    target_size=(224, 224),
    
    batch_size=batch_size,
    
    class_mode='categorical',
    
    subset='validation'
)

2023-04-28 03:30:49.516434: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 03:30:50.384652: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/apps/software/cuDNN/8.1.1.33-CUDA-11.2.2/lib:/opt/apps/software/CUDA/11.2.2/nvvm/lib64:/opt/apps/software/CUDA/11.2.2/extras/CUPTI/lib64:/opt/apps/software/CUDA/11.2.2/lib:/opt/apps/software/Python/3.7.13-GCCcore-11.2.0/lib:/opt/apps/software/OpenSSL/1.1/lib:/opt/apps/software/libffi/3.4.2-GCCcore-11.2.0/lib64:/opt/apps/software/GMP/6.2.1-GCCcore-11.2.0/lib:/opt/apps/software/XZ/5.2.5-GCCcore-

Found 22610 images belonging to 59 classes.
Found 5626 images belonging to 59 classes.


In [3]:
import tensorflow as tf

import tensorflow_hub as hub

import tensorflow.keras.regularizers as regularizers

from tfswin.model import SwinTransformerBase224

swin = SwinTransformerBase224(include_top=False)

model = tf.keras.Sequential([
    tf.keras.Input((224, 224, 3)),
    
    swin,
    
    Flatten(),
    
    Dense(512, activation='relu'),
    Dense(512, activation='relu', kernel_regularizer=regularizers.L2(0.01)),
    Dense(256, activation='relu'),
    Dense(256, activation='relu', kernel_regularizer=regularizers.L2(0.01)),
    Dense(128, activation='relu'),
    Dense(128, activation='relu', kernel_regularizer=regularizers.L2(0.01)),
    
    Dense(train_generator.num_classes, activation='softmax')
])

2023-04-28 03:30:52.444612: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 03:30:52.877001: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22284 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:41:00.0, compute capability: 8.6


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [5]:
num_samples = [320,
876,
1246,
300,
292,
698,
280,
280,
801,
868,
278,
278,
278,
278,
280,
284,
382,
254,
278,
278,
796,
286,
678,
300,
278,
282,
376,
288,
286,
1298,
684,
608,
962,
286,
384,
280,
1466,
278,
278,
1236,
286,
282,
280,
278,
280,
400,
302,
282,
280,
296,
406,
672,
287,
684,
1002,
284,
290,
1196,
290]

import numpy as np
total_classes = len(num_samples)
total_samples = np.sum(num_samples)
class_weights = {}

for i in range(len(num_samples)):
    class_weights[i] = total_samples/(total_classes * num_samples[i])
class_weights

{0: 1.4955508474576271,
 1: 0.5463199442767588,
 2: 0.38409010528606796,
 3: 1.5952542372881355,
 4: 1.6389598328302764,
 5: 0.6856393570006314,
 6: 1.7092009685230025,
 7: 1.7092009685230025,
 8: 0.5974734971116613,
 9: 0.5513551511364524,
 10: 1.7214973783684917,
 11: 1.7214973783684917,
 12: 1.7214973783684917,
 13: 1.7214973783684917,
 14: 1.7092009685230025,
 15: 1.6851277154452136,
 16: 1.2528174638388498,
 17: 1.8841585479781129,
 18: 1.7214973783684917,
 19: 1.7214973783684917,
 20: 0.6012264713397496,
 21: 1.6733436055469955,
 22: 0.7058647067646617,
 23: 1.5952542372881355,
 24: 1.7214973783684917,
 25: 1.6970789758384421,
 26: 1.2728092318788315,
 27: 1.661723163841808,
 28: 1.6733436055469955,
 29: 0.3687028283408634,
 30: 0.6996729110912875,
 31: 0.7871320249776985,
 32: 0.49748053137883647,
 33: 1.6733436055469955,
 34: 1.246292372881356,
 35: 1.7092009685230025,
 36: 0.3264503896224015,
 37: 1.7214973783684917,
 38: 1.7214973783684917,
 39: 0.38719763040974164,
 40: 1.67

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint

opt = tf.keras.optimizers.experimental.Nadam(learning_rate=0.0001)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1, mode='auto', min_lr=1e-8)

checkpoint_filepath = 'swin_best'

checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      mode='max',
                                      verbose=1)


model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=40,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // batch_size,
    verbose=1,
    callbacks=[reduce_lr, checkpoint_callback],
    shuffle=True,
    class_weight=class_weights
)

Epoch 1/40


2023-04-28 03:32:02.808309: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8101
2023-04-28 03:32:03.991725: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-04-28 03:32:04.544075: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x1e49c800 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-28 03:32:04.544128: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-04-28 03:32:04.553020: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-04-28 03:32:04.720009: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the 

706/706 [==============================] - ETA: 0s - loss: 7.9942 - accuracy: 0.5098
Epoch 1: val_accuracy improved from -inf to 0.77286, saving model to swin_best


INFO:tensorflow:Assets written to: swin_best/assets


INFO:tensorflow:Assets written to: swin_best/assets


...

INFO:tensorflow:Assets written to: swin_best/assets


706/706 [==============================] - 710s 1s/step - loss: 0.2707 - accuracy: 0.9899 - val_loss: 0.2916 - val_accuracy: 0.9886 - lr: 1.0000e-07
Epoch 33/40
706/706 [==============================] - ETA: 0s - loss: 0.2707 - accuracy: 0.9907
Epoch 33: val_accuracy did not improve from 0.98857
706/706 [==============================] - 625s 884ms/step - loss: 0.2707 - accuracy: 0.9907 - val_loss: 0.2963 - val_accuracy: 0.9862 - lr: 1.0000e-08
Epoch 34/40
706/706 [==============================] - ETA: 0s - loss: 0.2681 - accuracy: 0.9908
Epoch 34: val_accuracy did not improve from 0.98857
706/706 [==============================] - 624s 883ms/step - loss: 0.2681 - accuracy: 0.9908 - val_loss: 0.2942 - val_accuracy: 0.9855 - lr: 1.0000e-08
Epoch 35/40
706/706 [==============================] - ETA: 0s - loss: 0.2706 - accuracy: 0.9897
Epoch 35: val_accuracy improved from 0.98857 to 0.98911, saving model to swin_best


INFO:tensorflow:Assets written to: swin_best/assets


INFO:tensorflow:Assets written to: swin_best/assets


706/706 [==============================] - 870s 1s/step - loss: 0.2706 - accuracy: 0.9897 - val_loss: 0.2845 - val_accuracy: 0.9891 - lr: 1.0000e-08
Epoch 36/40
706/706 [==============================] - ETA: 0s - loss: 0.2704 - accuracy: 0.9901
Epoch 36: val_accuracy did not improve from 0.98911
706/706 [==============================] - 710s 1s/step - loss: 0.2704 - accuracy: 0.9901 - val_loss: 0.2974 - val_accuracy: 0.9873 - lr: 1.0000e-08
Epoch 37/40
706/706 [==============================] - ETA: 0s - loss: 0.2700 - accuracy: 0.9899
Epoch 37: val_accuracy did not improve from 0.98911
706/706 [==============================] - 623s 883ms/step - loss: 0.2700 - accuracy: 0.9899 - val_loss: 0.2879 - val_accuracy: 0.9889 - lr: 1.0000e-08
Epoch 38/40
706/706 [==============================] - ETA: 0s - loss: 0.2728 - accuracy: 0.9886
Epoch 38: val_accuracy did not improve from 0.98911
706/706 [==============================] - 624s 883ms/step - loss: 0.2728 - accuracy: 0.9886 - val_loss

In [10]:
import tensorflow as tf
test_model = tf.saved_model.load('swin_best')

In [11]:
import numpy as np

from tensorflow.keras.preprocessing import image

predictions = np.array([])

for i in range(2950):
    img = image.load_img('/common/scratch/projectgrps/CS424/assets/20230328/test_1/test_/' + str(i).zfill(6) + '.jpg', target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.
    
    probs = test_model(img_array)
    predicted_label = np.argmax(probs)
    predictions = np.append(predictions, predicted_label)

In [12]:
len(predictions)

2950

In [13]:
predictions

array([21., 38., 21., ..., 51., 54., 29.])

In [14]:
import pandas as pd

data_columns = ['id', 'category']

result_df = pd.DataFrame(columns=data_columns)

for i in range (len(predictions)):
    new_row = {'id': str(i).zfill(6), 'category': int(predictions[i])}
    result_df = result_df.append(new_row, ignore_index=True)
    
result_df.to_csv('result-swin.csv', index=False)

In [15]:
result_df

,id,category
0,000000,21
1,000001,38
2,000002,21
3,000003,47
4,000004,13
...,...,...
2945,002945,51
2946,002946,20
2947,002947,51
2948,002948,54
